In [1]:
from IPython.display import HTML
HTML('''
<script>
code_show=false;
function code_toggle(){
    if(code_show){$('.prompt, .input, .output_stderr, .output_error').hide();}
    else{$('.input, .prompt, .output_stderr, .output_error').show();}
    code_show=!code_show;
}
function initialize(){
    document.body.style.fontFamily='Palatino';
    var output=$('.output_subarea.output_text.output_stream.output_stdout');
    $.merge(output,$('.output_subarea.output_text.output_result'));
    for(var i=0;i<output.length;i++)for(var j=0;j<output[i].children.length;j++)
        output[i].children[j].style.fontFamily='Palatino';
    code_toggle();
}
$(document).ready(initialize);
</script>
Click <a href="javascript:code_toggle()">here</a> to show/hide codes in this notebook.
''')


In [2]:
    import spacy
    spacy.cli.download("en_core_web_md")
    import random
    import pickle
    from tensorflow import keras
    import gradio as gr
    import pandas as pd
    import numpy as np
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import gradio as gr

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
def get_poem(input_word, poet):
        
    #load all the data you need:
    if poet == 'neruda':
        model = keras.models.load_model('models/pn_model')
        word_index = pickle.load(open('pn_word_index.p','rb'))
        input_sequences = pickle.load(open('pn_input_sequences.p','rb'))
        max_sequence_len = max([len(x) for x in input_sequences])
        corpus_cleaned = pd.read_csv('pn_corpus.csv')
        tokenizer = pickle.load(open('pn_tokenizer.p','rb'))
    elif poet == 'oliver':
        model = keras.models.load_model('models/mo')
        word_index = pickle.load(open('mo_word_index.p','rb'))
        input_sequences = pickle.load(open('mo_input_sequences.p','rb'))
        max_sequence_len = max([len(x) for x in input_sequences])
        corpus_cleaned = pd.read_csv('mo_corpus.csv')
        tokenizer = pickle.load(open('mo_tokenizer.p','rb'))
    


        
    
    #input_word = input("Enter a word: ")
    nlp = spacy.load('en_core_web_md')
    topic = nlp(input_word)
    
    #Create a list for the similarities:
    sim_list = []
    rand_word_list=[]
    
    #choose 10 or so random words from word_index of data:
    for i in range(0, 20):
        rand_word = random.choice(list(word_index.keys()))
        #find word in spacy
        rand_word_spacy = nlp(rand_word)
        rand_word_list.append(rand_word)
    
        #Compute similarity for the word:
        similarity = (topic.similarity(rand_word_spacy))
        sim_list.append(similarity)
        
    #saves similarity to DataFrame
    df_1 = pd.DataFrame({'similarity' : sim_list, 'word' : rand_word_list })
    df_1.sort_values(by='similarity', inplace=True, ascending=False)
    
    
    #create input words
    words_to_use=['the ' + input_word]
    
    for i in range(0,2):
        words_to_use.append(df_1['word'][i])
        
    
        
    #Create the actual poem:
    poem = []
    
    choices = random.sample(words_to_use, 3)
    for item in choices: 
        seed_text = item
        next_words = random.choice([4,5,6,7,8])
        
        
    
        output_index = [0,0,0]
        output_word = [0,0,0]
        
        for _ in range(next_words):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
            predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
    
            
            # if-clauses I added, because otherwise the model sometimes gives the same word several times in a row:
            if predicted[0] == output_index[-1]:
                predicted = np.argsort(model.predict(token_list), axis=-1)[0][-2]
            
            # also this. Otherwise it sometimes gives a pair of two words several times in a row:
            if (predicted[0] == output_index[-2]) & (output_word[-1] == output_word[-3]):
                predicted = np.argsort(model_mo.predict(token_list), axis=-1)[0][-2]
            
            for word, index in tokenizer.word_index.items():
                if index == predicted:
                    output_word.append(word)
                    output_index.append(index)
                    break
            seed_text += " " + output_word[-1]
        
        poem.append(seed_text)
    return poem[0]+'\n'+ poem[1]+'\n'+ poem[2]

with gr.Blocks() as demo:
    name = gr.Textbox(label="Please enter a word:")
    output = gr.Textbox(label='Poem')
    poet = gr.Dropdown('oliver', 'neruda')
    poem_btn = gr.Button("Create a poem")
    poem_btn.click(fn=get_poem, inputs=(name, poet), outputs=output)

demo.launch(share=True)


#demo = gr.Interface(fn=get_poem, inputs="text", outputs="text")
#demo.launch(share=True)

TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given

In [4]:
HTML('''<script>initialize();</script>Click <a href="javascript:code_toggle()">here</a> to show/hide codes in this notebook.''')